# Generational Reduction and Diet Shift

This notebook quantifies generational changes in European diets using the cleaned GDD panel `data/processed/gdd_cleaned_panel.csv`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
DATA_FILE = Path('data/processed/gdd_cleaned_panel.csv')

In [ ]:
df = pd.read_csv(DATA_FILE)
cohort = df['year'] - ((df['age']//5)*5 + 2.5)
df['cohort'] = cohort

In [ ]:
def map_generation(cohort):
    if cohort < 1965:
        return 'Boomers'
    elif 1965 <= cohort <= 1980:
        return 'Gen X'
    elif 1981 <= cohort <= 1996:
        return 'Millennials'
    else:
        return 'Gen Z'

df['generation'] = df['cohort'].apply(map_generation)

In [ ]:
red_meat = df[df['food']=='Red meat'].copy()
summary = red_meat.groupby(['generation','age'])['median'].mean().reset_index()
age20 = summary[summary['age']==20].set_index('generation')['median']
age80 = summary[summary['age']==80].set_index('generation')['median']
reduction = ((age20 - age80)/age20*100).dropna()
print(reduction)

In [ ]:
fig, ax = plt.subplots()
for gen, group in summary.groupby('generation'):
    ax.plot(group['age'], group['median'], label=gen)
for gen, pct in reduction.items():
    ax.annotate(f'{pct:.1f}% drop', xy=(80, age80[gen]), textcoords='offset points', xytext=(-30,0))
ax.set_xlabel('Age')
ax.set_ylabel('Red meat intake (g/day)')
ax.legend()
fig.tight_layout()
Path('figures').mkdir(exist_ok=True)
fig.savefig('figures/meat_reduction_by_generation.png', dpi=300)
plt.close(fig)

In [ ]:
comp = df.groupby(['age','food'])['median'].mean().reset_index()
total = comp.groupby('age')['median'].transform('sum')
comp['share'] = comp['median']/total*100
foods = comp['food'].unique()
fig, ax = plt.subplots(figsize=(8,6))
bottom = np.zeros(comp['age'].nunique())
ages = sorted(comp['age'].unique())
for food in foods:
    vals = comp[comp['food']==food].set_index('age').reindex(ages)['share'].fillna(0)
    ax.bar(ages, vals, bottom=bottom, label=food)
    bottom += vals.values
ax.set_xlabel('Age')
ax.set_ylabel('Diet share (%)')
ax.legend(bbox_to_anchor=(1.05,1))
fig.tight_layout()
fig.savefig('figures/agegroup_diet_composition_stacked.png', dpi=300)
plt.close(fig)